In [1]:
!python -m pip install  torchvision pillow numpy opencv-python loguru pydantic

In [2]:
import time

import torch
from torchvision.models.detection import RetinaNet, retinanet_resnet50_fpn_v2
from pydantic import BaseModel
from typing import Protocol, TypeAlias, TypedDict
from numpy import ndarray
from loguru import logger

from utils.video import read_video, read_vid_batch


In [3]:
weights = "DEFAULT"
backend = "inductor"
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
model = retinanet_resnet50_fpn_v2(weights=weights).to(device)
model = torch.compile(model, backend=backend)
model.eval()

OptimizedModule(
  (_orig_mod): RetinaNet(
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05,

Process ForkProcess-6:
Process ForkProcess-8:
Process ForkProcess-4:
Process ForkProcess-5:
Traceback (most recent call last):
Process ForkProcess-7:
Process ForkProcess-1:
Process ForkProcess-3:
Process ForkProcess-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()


In [5]:
from utils.protocols import Detection


FrameId: TypeAlias = int
BoundingBox: TypeAlias = list[float]

SCORE_THRESHOLD: float = 0.5
PERSON_LABEL: int = 1
    


def pre_process_image(image_array: ndarray) -> torch.Tensor:
    image = image_array.transpose((-1, 0, 1))
    image = torch.from_numpy(image)
    image = image.to(device)
    image = image / 255
    image = image.unsqueeze(axis=0)
    return image



    return results

def detect(image_array: ndarray) -> list[Detection]:
    image = pre_process_image(image_array)
    with torch.no_grad():
        detections = model(image)
        detections = convert_model_detection(detections[0])
        #detections = post_process_detections(detections)
        return detections



In [6]:

from utils.protocols import Retina
from datetime import datetime
from utils.utils import convert_model_detection, get_gpu_name, get_file_name
import torch

MODEL="retina-net"
BASE_DIR="experiments/retina"

def process_video(vid_file, frames=[]):
    start_time = datetime.now().isoformat()
    start = time.time()
    results = {}
    for frame_id, frame in read_video(vid_file):
        results[frame_id] = detect(frame)
    end = time.time()
    end_time = datetime.now().isoformat()
    n_frames = frame_id+1 if not frames else len(frames)

    exp = Retina(
        model="retinanet",
        gpu=get_gpu_name(),
        filename=get_file_name(BASE_DIR, start, MODEL, vid_file),
        file=vid_file,
        frames=None if not frames else frames,
        n_frames=n_frames,
        processing_time=end-start,
        fps=n_frames/(end-start),
        data=results,
        start_time=start_time,
        end_time=end_time
    )
    exp.save()
    exp.log()
    torch.cuda.empty_cache()
    return exp


In [7]:
results = []
for i in range(5):
    vid_hd = process_video("data/720.mp4")
    results.append(vid_hd)
    vid_fhd = process_video("data/1080.mp4")
    results.append(vid_fhd)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/overrides.py:110: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  torch.has_cuda,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/overrides.py:111: UserWarning: 'has_cudnn' is deprecated, please use 'torch.backends.cudnn.is_available()'
  torch.has_cudnn,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/overrides.py:117: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  torch.has_mps,
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/overrides.py:118: UserWarning: 'has_mkldnn' is deprecated, please use 'torch.backends.mkldnn.is_available()'
  torch.has_mkldnn,
2024-06-11 10:43:00.366 | INFO     | utils.protocols:log:64 - data/720.mp4 | frames=283 | delta=84.62799668312073 | fps=3.3440470186203175
2024-06-11 10:43:39.769 | INFO     | utils.protocols:log:64

In [8]:
columns = results[0].columns
rows = [result.row for result in results]

In [9]:
import pandas as pd

df = pd.DataFrame(rows, columns=columns)
df.head()

,model,gpu,file,batch_size,n_frames,processing_time,fps,start_time,end_time,result_file,data
0,retinanet,Tesla V100-SXM2-16GB,data/720.mp4,1,283,84.627997,3.344047,2024-06-11T10:41:34.767781,2024-06-11T10:42:59.395793,experiments/retina/17181024947677927-retina-ne...,"{0: [box=[419.6314697265625, 96.96260833740234..."
1,retinanet,Tesla V100-SXM2-16GB,data/1080.mp4,1,283,38.452501,7.359729,2024-06-11T10:43:00.376270,2024-06-11T10:43:38.828788,experiments/retina/17181025803762827-retina-ne...,"{0: [box=[637.8360595703125, 142.39697265625, ..."
2,retinanet,Tesla V100-SXM2-16GB,data/720.mp4,1,283,13.876576,20.394080,2024-06-11T10:43:39.779517,2024-06-11T10:43:53.656109,experiments/retina/17181026197795293-retina-ne...,"{0: [box=[6.1236742890994506e-43, 1.4153114489..."
3,retinanet,Tesla V100-SXM2-16GB,data/1080.mp4,1,283,13.506378,20.953064,2024-06-11T10:43:54.758967,2024-06-11T10:44:08.265362,experiments/retina/171810263475898-retina-net-...,"{0: [box=[637.8360595703125, 142.39697265625, ..."
4,retinanet,Tesla V100-SXM2-16GB,data/720.mp4,1,283,13.017989,21.739149,2024-06-11T10:44:09.215048,2024-06-11T10:44:22.233058,experiments/retina/1718102649215065-retina-net...,"{0: [box=[6.1236742890994506e-43, 1.4153114489..."


In [10]:
from datetime import datetime
now = datetime.now()
csv_file = f"__{MODEL}-{get_gpu_name().replace(' ','_')}-{now.day}-{now.hour}-{now.min}.csv"
df.to_csv(csv_file)